Referência: https://github.com/prashil2792/Question-Answering-System-Deep-Learning/blob/master/Question%20Answering%20Memory%20Network.ipynb

In [ ]:
import pandas as pd
import numpy as np
import random
from tensorflow import keras
import tensorflow as tf
import tensorflow_addons as tfa
import keras_tuner as kt
from sklearn.metrics import classification_report
import datetime
import os

from classes import Preprocessing, Model, Lstm, Similarity, Bert, SentenceTransformers
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import LSTM, GRU
from tensorflow.keras.layers import Embedding
from keras.layers import add, dot, concatenate
from keras.layers import Input, Activation, Dense, Permute, Dropout

SEED_VAL = 42
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
tf.random.set_seed(SEED_VAL)

In [81]:
df = pd.read_csv("data/pokemon_contexts2.csv", sep=',')  

df['answers'] = df['answers']
df.head()

,id,title,context,question,answers
0,2,Bulbasaur,Bulbasaur its a Grass first generation Pokemon...,Whats the type of Bulbasaur?,Grass
1,3,Ivysaur,Ivysaur its a Grass first generation Pokemon t...,Whats the type of Ivysaur?,Grass
2,4,Venusaur,Venusaur its a Grass first generation Pokemon ...,Whats the type of Venusaur?,Grass
3,5,Charmander,Charmander its a Fire first generation Pokemon...,Whats the type of Charmander?,Fire
4,6,Charmeleon,Charmeleon its a Fire first generation Pokemon...,Whats the type of Charmeleon?,Fire


In [82]:
preprocessing = Preprocessing()

pp_df, corpus_pp_desc_q, corpus_pp_desc_a = Preprocessing.apply_preprocessing(df)

pp_df.head()

,id,title,context,question,answers
0,2,Bulbasaur,Bulbasaur its a Grass first generation Pokemon...,"[whats, the, type, of, bulbasaur, ?]",[grass]
1,3,Ivysaur,Ivysaur its a Grass first generation Pokemon t...,"[whats, the, type, of, ivysaur, ?]",[grass]
2,4,Venusaur,Venusaur its a Grass first generation Pokemon ...,"[whats, the, type, of, venusaur, ?]",[grass]
3,5,Charmander,Charmander its a Fire first generation Pokemon...,"[whats, the, type, of, charmander, ?]",[fire]
4,6,Charmeleon,Charmeleon its a Fire first generation Pokemon...,"[whats, the, type, of, charmeleon, ?]",[fire]


In [83]:
df_train, df_val, df_test = preprocessing.split_dataset(df=pp_df, columns=['question'], label='answers', seed=SEED_VAL)

df_train.head()

train: 70%
val: 10%
test: 21%


,question,answers
59,"[whats, the, type, of, poliwag, ?]",[water]
116,"[whats, the, type, of, seadra, ?]",[water]
15,"[whats, the, type, of, pidgey, ?]",[normal]
61,"[whats, the, type, of, poliwrath, ?]",[water]
1,"[whats, the, type, of, ivysaur, ?]",[grass]


In [180]:
mean_sequence_length_q, max_sequence_length_q = Preprocessing.get_sequences_details(df_train['question'])
mean_sequence_length_a, max_sequence_length_a = Preprocessing.get_sequences_details(df_train['answers'])

print("Question:")
print(f'Mean sequence length: {mean_sequence_length_q}')
print(f'Max sequence length: {max_sequence_length_q}')

print("Answer:")
print(f'Mean sequence length: {mean_sequence_length_a}')
print(f'Max sequence length: {max_sequence_length_a}')

Question:
Mean sequence length: 6.0
Max sequence length: 6
Answer:
Mean sequence length: 1.0
Max sequence length: 1


In [181]:
MAX_SEQUENCE_LENGTH = max_sequence_length_q if max_sequence_length_q > max_sequence_length_a else max_sequence_length_a

vocab_size_q, X_train_padded, X_val_padded, X_test_padded = Preprocessing.adapt_X_for_input_layer(df_train['question'].astype(str), df_val['question'].astype(str), df_test['question'].astype(str), MAX_SEQUENCE_LENGTH)
vocab_size_a, Y_train_padded, Y_val_padded, Y_test_padded = Preprocessing.adapt_X_for_input_layer(df_train['answers'].astype(str), df_val['answers'].astype(str), df_test['answers'].astype(str), MAX_SEQUENCE_LENGTH)

print("Question:")
print('Training features shape:', X_train_padded.shape)
print('Validation features shape:', X_val_padded.shape)
print('Test features shape:', X_test_padded.shape)

print("Answer:")
print('Training features shape:', Y_train_padded.shape)
print('Validation features shape:', Y_val_padded.shape)
print('Test features shape:', Y_test_padded.shape)

X_train_padded[:10]

Question:
Training features shape: (105, 6)
Validation features shape: (15, 6)
Test features shape: (31, 6)
Answer:
Training features shape: (105, 6)
Validation features shape: (15, 6)
Test features shape: (31, 6)


array([[ 3,  4,  5,  6,  1,  1],
       [ 3,  4,  5,  7,  1,  1],
       [ 3,  4,  5,  8,  1,  1],
       [ 3,  4,  5,  9,  1,  1],
       [ 3,  4,  5, 10,  1,  1],
       [ 3,  4,  5, 11,  1,  1],
       [ 3,  4,  5, 12,  1,  1],
       [ 3,  4,  5, 13,  1,  1],
       [ 3,  4,  5, 14,  1,  1],
       [ 3,  4,  5, 15,  1,  1]], dtype=int32)

In [182]:
df_train['answers'].value_counts()

answers
[water]       23
[normal]      11
[fire]        10
[grass]        9
[bug]          9
[poison]       9
[rock]         7
[psychic]      7
[electric]     5
[ground]       4
[fighting]     4
[ghost]        3
[fairy]        2
[ice]          1
[dragon]       1
Name: count, dtype: int64

In [163]:
VOCAB_SIZE = vocab_size_q if vocab_size_q > vocab_size_a else vocab_size_a
METRICS = ['accuracy']

In [382]:
question_list = list(pp_df['question'].apply(lambda x: ' '.join(x)))
answers_list = list(pp_df['answers'].apply(lambda x: ' '.join(x)))

question_words = [word for sentence in question_list for word in sentence.split()]
answers_words = [word for sentence in answers_list for word in sentence.split()]

# # Remove duplicates and create the vocabulary list
vocabulary_list = sorted(list(set(question_words + answers_words)))

VOCAB_SIZE = len(vocabulary_list)+1
print(VOCAB_SIZE)
vocabulary_list

173


["'d",
 '?',
 'abra',
 'aerodactyl',
 'alakazam',
 'arbok',
 'arcanine',
 'articuno',
 'beedrill',
 'bellsprout',
 'blastoise',
 'bug',
 'bulbasaur',
 'butterfree',
 'caterpie',
 'chansey',
 'charizard',
 'charmander',
 'charmeleon',
 'clefable',
 'clefairy',
 'cloyster',
 'cubone',
 'dewgong',
 'diglett',
 'ditto',
 'dodrio',
 'doduo',
 'dragon',
 'dragonair',
 'dragonite',
 'dratini',
 'drowzee',
 'dugtrio',
 'eevee',
 'ekans',
 'electabuzz',
 'electric',
 'electrode',
 'exeggcute',
 'exeggutor',
 'fairy',
 'farfetch',
 'fearow',
 'fighting',
 'fire',
 'flareon',
 'gastly',
 'gengar',
 'geodude',
 'ghost',
 'gloom',
 'golbat',
 'goldeen',
 'golduck',
 'golem',
 'grass',
 'graveler',
 'grimer',
 'ground',
 'growlithe',
 'gyarados',
 'haunter',
 'hitmonchan',
 'hitmonlee',
 'horsea',
 'hypno',
 'ice',
 'ivysaur',
 'jigglypuff',
 'jolteon',
 'jynx',
 'kabuto',
 'kabutops',
 'kadabra',
 'kakuna',
 'kangaskhan',
 'kingler',
 'koffing',
 'krabby',
 'lapras',
 'lickitung',
 'machamp',
 'mac

In [383]:
word_idx = dict((c, i + 1) for i, c in enumerate(vocabulary_list))
idx_word = dict((i+1, c) for i,c in enumerate(vocabulary_list))

In [390]:
def vectorize_phrases(data, word_idx, question_maxlen):
    X = []
    Y = []
    for idx, row in data.iterrows():
        x = [word_idx[w] for w in row['question']]
        # y = np.zeros(len(word_idx) + 1)
        # y = [word_idx[w] for w in row['answers']]
        y = np.zeros(len(word_idx) + 1)
        y[word_idx[row['answers'][0]]] = 1
        # y[word_idx[row['answers'][0]]] = 1
        X.append(x)
        Y.append(y)
    return (pad_sequences(X, maxlen=question_maxlen), np.array(Y))

question_train, answers_train = vectorize_phrases(df_train,
                                                word_idx,
                                                VOCAB_SIZE)
question_val, answers_val = vectorize_phrases(df_val,
                                                word_idx,
                                                VOCAB_SIZE)
question_test, answers_test = vectorize_phrases(df_test,
                                                word_idx,
                                                VOCAB_SIZE)

In [391]:
question_train.shape, answers_train.shape

((105, 173), (105, 173))

In [403]:
# placeholders
MAX_SEQUENCE_LENGTH = VOCAB_SIZE
input_sequence = Input((MAX_SEQUENCE_LENGTH,))

print('Input sequence:', input_sequence)
# print('Question:', question)

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size_q,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)



# embed the input into a sequence of vectors of size MAX_SEQUENCE_LENGTH
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size_q,
                              output_dim=MAX_SEQUENCE_LENGTH))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, MAX_SEQUENCE_LENGTH)



# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size_q,
                               output_dim=64,
                               input_length=MAX_SEQUENCE_LENGTH))

question_encoder.add(Dropout(0.3))
# output: (samples, MAX_SEQUENCE_LENGTH, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
print('Input encoded m', input_encoded_m)
input_encoded_c = input_encoder_c(input_sequence)
print('Input encoded c', input_encoded_c)
question_encoded = question_encoder(input_sequence)
print('Question encoded', question_encoded)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, MAX_SEQUENCE_LENGTH)
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
print(match.shape)
match = Activation('softmax')(match)
print('Match shape', match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, MAX_SEQUENCE_LENGTH)
response = Permute((2, 1))(response)  # (samples, MAX_SEQUENCE_LENGTH, story_maxlen)
print('Response shape', response)

# concatenate the response vector with the question vector sequence
answer = concatenate([response, question_encoded])
print('Answer shape', answer)

#answer = LSTM(lstm_size, return_sequences=True)(answer)  # Generate tensors of shape 32
#answer = Dropout(0.3)(answer)
LSTM_UNITS = 64
answer = LSTM(LSTM_UNITS)(answer)  # Generate tensors of shape 32
answer = Dropout(0.3)(answer)
answer = Dense(VOCAB_SIZE)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

Input sequence: KerasTensor(type_spec=TensorSpec(shape=(None, 173), dtype=tf.float32, name='input_27'), name='input_27', description="created by layer 'input_27'")
Input encoded m KerasTensor(type_spec=TensorSpec(shape=(None, 173, 64), dtype=tf.float32, name=None), name='sequential_59/dropout_66/Identity:0', description="created by layer 'sequential_59'")
Input encoded c KerasTensor(type_spec=TensorSpec(shape=(None, 173, 173), dtype=tf.float32, name=None), name='sequential_60/dropout_67/Identity:0', description="created by layer 'sequential_60'")
Question encoded KerasTensor(type_spec=TensorSpec(shape=(None, 173, 64), dtype=tf.float32, name=None), name='sequential_61/dropout_68/Identity:0', description="created by layer 'sequential_61'")
(None, 173, 173)
Match shape KerasTensor(type_spec=TensorSpec(shape=(None, 173, 173), dtype=tf.float32, name=None), name='activation_24/Softmax:0', description="created by layer 'activation_24'")
Response shape KerasTensor(type_spec=TensorSpec(shape=(N

2023-07-02 03:00:40.023119: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-07-02 03:00:40.030435: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-07-02 03:00:40.037714: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [404]:
# build the final model
model = Model(input_sequence, answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [372]:
# tuner = kt.RandomSearch(
#     hypermodel=Lstm(vocab_size_q, MAX_SEQUENCE_LENGTH, METRICS, None),
#     objective=kt.Objective('accuracy', direction='max'),
#     max_trials=1,
#     executions_per_trial=1,
#     # overwrite=True,
#     directory="hyperparameters_search",
#     project_name="model",
#     seed=SEED_VAL
# )

# print(tuner.search_space_summary())

# logdir = os.path.join("logs/model/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
# tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)
# earlystopping_callback = keras.callbacks.EarlyStopping('val_loss', mode='min', verbose=1, patience=5)
# # visualizer_callback = TrainingVisualizer()

# callbacks_list = [earlystopping_callback, tensorboard_callback] #, visualizer_callback]

In [373]:
question_train

array([[  0,   0,   0, ..., 108, 121,   2],
       [  0,   0,   0, ..., 108, 139,   2],
       [  0,   0,   0, ..., 108, 117,   2],
       ...,
       [  0,   0,   0, ..., 108,  48,   2],
       [  0,   0,   0, ..., 108,  67,   2],
       [  0,   0,   0, ..., 108, 105,   2]], dtype=int32)

In [374]:
answers_train

array([[165],
       [165],
       [106],
       [165],
       [ 57],
       [ 46],
       [ 60],
       [ 51],
       [ 12],
       [135],
       [127],
       [120],
       [106],
       [106],
       [120],
       [165],
       [165],
       [106],
       [ 12],
       [ 46],
       [120],
       [ 38],
       [ 60],
       [ 51],
       [120],
       [ 38],
       [ 57],
       [ 12],
       [ 57],
       [106],
       [165],
       [ 45],
       [165],
       [165],
       [ 12],
       [165],
       [165],
       [ 12],
       [106],
       [120],
       [127],
       [ 45],
       [120],
       [ 46],
       [165],
       [106],
       [120],
       [ 45],
       [165],
       [ 46],
       [ 46],
       [106],
       [ 60],
       [106],
       [135],
       [165],
       [ 12],
       [ 60],
       [165],
       [135],
       [ 46],
       [ 42],
       [ 46],
       [ 57],
       [127],
       [127],
       [ 57],
       [ 45],
       [ 38],
       [120],
       [135],
      

In [405]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_27 (InputLayer)          [(None, 173)]        0           []                               
                                                                                                  
 sequential_59 (Sequential)     (None, None, 64)     7168        ['input_27[0][0]']               
                                                                                                  
 sequential_61 (Sequential)     (None, 173, 64)      7168        ['input_27[0][0]']               
                                                                                                  
 dot_16 (Dot)                   (None, 173, 173)     0           ['sequential_59[0][0]',          
                                                                  'sequential_61[0][0]']   

In [395]:
batch_size = 6
train_epochs = 4

In [396]:
question_train.shape

(105, 173)

In [397]:
answers_train.shape

(105, 173)

In [398]:
question_val.shape

(15, 173)

In [399]:
answers_val.shape

(15, 173)

In [406]:
model.fit(question_train, answers_train, batch_size, train_epochs,
          validation_data=(question_val, answers_val))

Epoch 1/4


2023-07-02 03:00:54.657573: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [105,173]
	 [[{{node Placeholder/_11}}]]
2023-07-02 03:00:54.659620: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [105,173]
	 [[{{node Placeholder/_11}}]]
2023-07-02 03:00:55.992385: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/spl

InvalidArgumentError: Graph execution error:

Detected at node 'model_12/sequential_59/embedding_57/embedding_lookup' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/mayara/Documentos/unb/PAA/Q&A/Modelo-Paa/new_model/.venv/lib/python3.8/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/mayara/Documentos/unb/PAA/Q&A/Modelo-Paa/new_model/.venv/lib/python3.8/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/mayara/Documentos/unb/PAA/Q&A/Modelo-Paa/new_model/.venv/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 728, in start
      self.io_loop.start()
    File "/home/mayara/Documentos/unb/PAA/Q&A/Modelo-Paa/new_model/.venv/lib/python3.8/site-package

In [294]:
# tuner.search(question_train, answers_train,
#              validation_data=(question_val, answers_val),
#              callbacks=callbacks_list)

INFO:tensorflow:Oracle triggered exit


In [ ]:
X_train_padded.shape

In [ ]:
Y_train_padded.shape

In [ ]:
best_model = tuner.get_best_models()[0]
Model.save(best_model, 'model')

best_hps = tuner.get_best_hyperparameters()[0]
print(best_hps.values)

In [ ]:
best_model.summary()

In [ ]:
hypermodel = Model.recover('model')

EPOCHS = best_hps['epochs']
BATCH_SIZE = best_hps['batch_size']
LEARNING_RATE = best_hps['learning_rate']
LOSS = 'categorical_crossentropy'
METRICS = ['accuracy']

hypermodel.compile(ptimizer='rmsprop',
                    loss=LOSS, 
                    metrics=METRICS)

In [ ]:
eval_result_val = hypermodel.evaluate(X_val_padded, Y_val_padded)

y_probabilities_val = hypermodel.predict(X_val_padded)
y_pred_val = np.argmax(y_probabilities_val, axis=1)
y_val = np.argmax(Y_val_padded, axis=1)

print('\nValidation')
print(classification_report(y_val, y_pred_val, zero_division=True))

Save results

In [ ]:
eval_result_test = hypermodel.evaluate(X_test_padded, Y_test_padded)

y_probabilities_test = hypermodel.predict(X_test_padded)
y_pred_test = np.argmax(y_probabilities_test, axis=1)
y_test = np.argmax(Y_test_padded, axis=1)

print('\nTest')
print(classification_report(y_test, y_pred_test, zero_division=True))